In [1]:
%pip install pandas
%pip install pycurl
%pip install selenium

import pandas as pd
import urllib.request
import zipfile
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
url = 'https://data.binance.vision/?prefix=data/spot/monthly/klines/'+market+'/' + timeframe +'/'+market+'-'+timeframe+'-'+a+'.zip'

NameError: name 'market' is not defined

In [5]:
#(1.01)^n*(0.99)^(1000-n)-1  die Formel hat ergeben dass man möglcihst einen niedrigen hebel verwendet damit man sich mehr fehler erlauben kann
#man mmuss dann anhand der erfolgswwahrscheinlichkeit(n/m) (n... anzahl der geklappten versuche  m.... anzahl der Versuche) schauen was sich mit steigenden m mehr lohnt  

In [3]:
df = pd.DataFrame([[0,0,0,0,0]])

#enddate='2024-03'
enddate='2024-08'

#die Funktion bekommt endzeitpunkt so wie den Markt und die timeframe  (ob 15m, 1h, 2h ..) und gibt den Datensatz wieder von allen
#Daten die Binance liefert
def datag(df,market,timeframe,enddate):
    PATH="C:/Program Files (x86)/ChromeDriver/chromedriver.exe"
    driver = webdriver.Chrome()
    
    
    driver.get("https://data.binance.vision/?prefix=data/futures/um/monthly/klines/"+market+"/"+timeframe+"/")
    try:
        main=driver.find_element(By.ID, "listing")
        time.sleep(2)
    except:
        driver.quit
    first_paragraph = ""
    for i in range(len(main.text)-1, -1, -1):
        if main.text[i] == "\n":
                # Absatz gefunden
                break
        else:
            first_paragraph += main.text[i]
    
    # Umkehren der Zeichenkette
    first_paragraph = first_paragraph[::-1]
    # Speichern der ersten Spalte
    first_paragraph
    
    
    index = first_paragraph.find(timeframe+"-")
    if index != -1:
        startdate = first_paragraph[index + len(timeframe+"-"):index + len(timeframe+"-")+7]
       #startdate="2024-06"
    else:
        print("Zeichenfolge '15m-' nicht gefunden")
    
    start_date = pd.to_datetime(startdate)
    end_date = pd.to_datetime(enddate)
    dates = pd.date_range(start_date, end_date, freq="ME")

    # Alternative with list comprehension
    formatted_dates = [d.strftime("%Y-%m") for d in dates] + [enddate]
    for a in formatted_dates:
        url = 'https://data.binance.vision/data/futures/um/monthly/klines/'+market+'/' + timeframe +'/'+ market+'-'+timeframe+'-'+ a +'.zip'
        urllib.request.urlretrieve(url, 'Data.zip')

        zip_file = "Data.zip"

        # Zielverzeichnis zum Entpacken
        target_dir = "C:/Users/Dave/work"

        #Überprüfen ob es eine csv in dem Ordner gibt - wenn ja removen
        if (os.path.isfile(os.path.join(target_dir,"data.csv"))):
            os.remove(os.path.join(target_dir,"data.csv"))
    
        # Extrahieren der Datei
        with zipfile.ZipFile(zip_file, "r") as z:
          for filename in z.namelist():
            z.extract(filename, target_dir, f"newdata.xlsx")
        os.rename(os.path.join(target_dir, filename), os.path.join(target_dir, "data.csv"))

        dftemp=pd.read_csv("data.csv",usecols=[0,1,2,3,4],header=None,names=None)
       
        df= pd.concat([df, dftemp], axis=0, sort=False,ignore_index=True,join='outer')
    df=df.iloc[1:]
    # df = df.iloc[:, 1:]
    df=df.reset_index(drop=True)
    df.columns=["time_open","open","high","low","close"]
    df=df.loc[df['open'] != "open"]
    df=df.reset_index(drop=True)
    return df
    
    

In [6]:
dfa=datag(df,"BTCUSDT","1h",enddate)

In [4]:

dfa
pfad = "F:/Data/5m_data_every_market/"
dateiname ="5mBTCUSDT.csv"
dfa.to_csv(os.path.join(pfad, dateiname), index=False)

In [31]:
unix_timestamp=dfa["time_open"][0]
from datetime import datetime


# Konvertiere in datetime-Objekt

time_struct = time.localtime(unix_timestamp/1000)

# Formatiere das Datum (verschiedene Optionen verfügbar)
date_string = time.strftime("%Y-%m-%d", time_struct)  # Format: JJJJ-MM-TT
print(date_string)  # Ausgabe: 2024-02-01

2020-01-01


In [30]:
time.localtime(1577836800)

time.struct_time(tm_year=2020, tm_mon=1, tm_mday=1, tm_hour=1, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=1, tm_isdst=0)

In [3]:
timeframes=["1m","3m","5m","15m","30m","1h","2h","4h","12h","1d"]


pfad = "C:/Users/Dave/work/SOLUSDT-data"

# Ordner erstellen, falls er nicht existiert
if not os.path.exists(pfad):
    os.makedirs(pfad)

# Dataframe als CSV-Datei speichern


for i in timeframes:
    df = pd.DataFrame([[0,0,0,0]])
    # Dateiname angeben
    dateiname = i+"SOLUSDT"+".csv"
    df=datag(df,"SOLUSDT",i,enddate)
    df.to_csv(os.path.join(pfad, dateiname), index=False)

In [34]:
from binance.client import Client

api_key = "xxx"
api_secret = "xxx"
client = Client(api_key, api_secret)
exchange_info = client.futures_exchange_info()

In [38]:
for s in exchange_info['symbols']:
    print(s['symbol'])


BTCUSDT
ETHUSDT
BCHUSDT
XRPUSDT
EOSUSDT
LTCUSDT
TRXUSDT
ETCUSDT
LINKUSDT
XLMUSDT
ADAUSDT
XMRUSDT
DASHUSDT
ZECUSDT
XTZUSDT
BNBUSDT
ATOMUSDT
ONTUSDT
IOTAUSDT
BATUSDT
VETUSDT
NEOUSDT
QTUMUSDT
IOSTUSDT
THETAUSDT
ALGOUSDT
ZILUSDT
KNCUSDT
ZRXUSDT
COMPUSDT
OMGUSDT
DOGEUSDT
SXPUSDT
KAVAUSDT
BANDUSDT
RLCUSDT
WAVESUSDT
MKRUSDT
SNXUSDT
DOTUSDT
DEFIUSDT
YFIUSDT
BALUSDT
CRVUSDT
TRBUSDT
RUNEUSDT
SUSHIUSDT
SRMUSDT
EGLDUSDT
SOLUSDT
ICXUSDT
STORJUSDT
BLZUSDT
UNIUSDT
AVAXUSDT
FTMUSDT
HNTUSDT
ENJUSDT
FLMUSDT
TOMOUSDT
RENUSDT
KSMUSDT
NEARUSDT
AAVEUSDT
FILUSDT
RSRUSDT
LRCUSDT
MATICUSDT
OCEANUSDT
CVCUSDT
BELUSDT
CTKUSDT
AXSUSDT
ALPHAUSDT
ZENUSDT
SKLUSDT
GRTUSDT
1INCHUSDT
CHZUSDT
SANDUSDT
ANKRUSDT
BTSUSDT
LITUSDT
UNFIUSDT
REEFUSDT
RVNUSDT
SFPUSDT
XEMUSDT
BTCSTUSDT
COTIUSDT
CHRUSDT
MANAUSDT
ALICEUSDT
HBARUSDT
ONEUSDT
LINAUSDT
STMXUSDT
DENTUSDT
CELRUSDT
HOTUSDT
MTLUSDT
OGNUSDT
NKNUSDT
SCUSDT
DGBUSDT
1000SHIBUSDT
BAKEUSDT
GTCUSDT
BTCDOMUSDT
IOTXUSDT
AUDIOUSDT
RAYUSDT
C98USDT
MASKUSDT
ATAUSDT
DYDXUSDT
1000XECUSD

In [5]:
from binance.client import Client

api_key = "xxx"
api_secret = "xxx"
client = Client(api_key, api_secret)
exchange_info = client.futures_exchange_info()
#exchange_info = client.get_exchange_info()
pfad = "F:/Data/1m_data_every_market"
count=0
for s in exchange_info['symbols']:
    if (count>46):
        df = pd.DataFrame([[0,0,0,0,0]])
        df=datag(df,s['symbol'],"1m",enddate)
        dateiname = "1m"+s['symbol']+".csv"
        df.to_csv(os.path.join(pfad, dateiname), index=False)
        print(count)
    count=count+1

47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
3

UnboundLocalError: cannot access local variable 'startdate' where it is not associated with a value

In [53]:
from binance.client import Client

api_key = "xxx"
api_secret = "xxx"
client = Client(api_key, api_secret)
exchange_info = client.futures_exchange_info()
#exchange_info = client.get_exchange_info()
pfad = "F:/Data/1h_data_every_market"
count=0
#for s in exchange_info['symbols']:
df = pd.DataFrame([[0,0,0,0,0]])
df=datag(df,"ETHUSDT_240329","1h",enddate)
dateiname = "1h"+"ETHUSDT_240329"+".csv"
df.to_csv(os.path.join(pfad, dateiname), index=False)
print(count)
count=count+1

0


In [7]:
api_key = "xxx"
api_secret = "xxx"
client = Client(api_key, api_secret)
#exchange_info = client.futures_exchange_info()
exchange_info = client.get_exchange_info()

In [20]:
count=0
for s in exchange_info['symbols']:
    if s['symbol'].endswith('USDT'):
         count=count+1
         print(s['symbol'])

BTCUSDT
ETHUSDT
BNBUSDT
BCCUSDT
NEOUSDT
LTCUSDT
QTUMUSDT
ADAUSDT
XRPUSDT
EOSUSDT
TUSDUSDT
IOTAUSDT
XLMUSDT
ONTUSDT
TRXUSDT
ETCUSDT
ICXUSDT
VENUSDT
NULSUSDT
VETUSDT
PAXUSDT
BCHABCUSDT
BCHSVUSDT
USDCUSDT
LINKUSDT
WAVESUSDT
BTTUSDT
USDSUSDT
ONGUSDT
HOTUSDT
ZILUSDT
ZRXUSDT
FETUSDT
BATUSDT
XMRUSDT
ZECUSDT
IOSTUSDT
CELRUSDT
DASHUSDT
NANOUSDT
OMGUSDT
THETAUSDT
ENJUSDT
MITHUSDT
MATICUSDT
ATOMUSDT
TFUELUSDT
ONEUSDT
FTMUSDT
ALGOUSDT
USDSBUSDT
GTOUSDT
ERDUSDT
DOGEUSDT
DUSKUSDT
ANKRUSDT
WINUSDT
COSUSDT
NPXSUSDT
COCOSUSDT
MTLUSDT
TOMOUSDT
PERLUSDT
DENTUSDT
MFTUSDT
KEYUSDT
STORMUSDT
DOCKUSDT
WANUSDT
FUNUSDT
CVCUSDT
CHZUSDT
BANDUSDT
BUSDUSDT
BEAMUSDT
XTZUSDT
RENUSDT
RVNUSDT
HCUSDT
HBARUSDT
NKNUSDT
STXUSDT
KAVAUSDT
ARPAUSDT
IOTXUSDT
RLCUSDT
MCOUSDT
CTXCUSDT
BCHUSDT
TROYUSDT
VITEUSDT
FTTUSDT
EURUSDT
OGNUSDT
DREPUSDT
BULLUSDT
BEARUSDT
ETHBULLUSDT
ETHBEARUSDT
TCTUSDT
WRXUSDT
BTSUSDT
LSKUSDT
BNTUSDT
LTOUSDT
EOSBULLUSDT
EOSBEARUSDT
XRPBULLUSDT
XRPBEARUSDT
STRATUSDT
AIONUSDT
MBLUSDT
COTIUSDT
BNBBULLUSDT
BNB

In [21]:
count

493

In [80]:
dfa= pd.concat([df, dftemp], axis=0, sort=False,ignore_index=True,join='outer')

In [74]:
dfa=dfa.iloc[1:]
dfa = dfa.iloc[:, 1:]
dfa=dfa.reset_index(drop=True)

In [76]:
dfa.columns=["open","high","low","close"]

In [81]:
dfa= pd.concat([dfa, dftemp], axis=0, sort=False,ignore_index=True,join='outer')

In [20]:
dfa

,open,high,low,close
0,7189.43,7190.52,7170.15,7171.55
1,7171.43,7225.0,7171.1,7210.24
2,7210.38,7239.3,7206.46,7237.99
3,7237.41,7239.74,7215.0,7221.65
4,7221.8,7225.41,7211.22,7213.86
...,...,...,...,...
36403,60921.40,61730.80,60727.60,61681.80
36404,61681.70,62345.70,61598.70,61974.60
36405,61974.60,62038.60,60619.70,61435.80
36406,61435.80,61541.20,60722.40,61290.50


In [14]:
dfa=dfa.reset_index(drop=True)
dfa.columns=["open","high","low","close"]
dfa=dfa.loc[dfa['open'] != "open"]
dfa=dfa.reset_index(drop=True)
dfa

,open,high,low,close
0,7189.43,7190.52,7170.15,7171.55
1,7171.43,7225.00,7171.10,7210.24
2,7210.38,7239.30,7206.46,7237.99
3,7237.41,7239.74,7215.00,7221.65
4,7221.80,7225.41,7211.22,7213.86
...,...,...,...,...
739,9330.28,9335.44,9310.42,9316.86
740,9316.76,9355.33,9316.76,9338.50
741,9338.28,9382.91,9331.00,9375.93
742,9375.94,9438.00,9351.00,9415.24


In [92]:
dfa[dfa["open"]=="open"]

,open,high,low,close
70175,open,high,low,close
73152,open,high,low,close
75841,open,high,low,close
78818,open,high,low,close
81699,open,high,low,close
84292,open,high,low,close
87173,open,high,low,close
90150,open,high,low,close
93127,open,high,low,close
96008,open,high,low,close


In [24]:
df = pd.DataFrame([[0,0,0,0]])

#enddate='2024-02'
enddate='2024-02'

#die Funktion bekommt endzeitpunkt so wie den Markt und die timeframe  (ob 15m, 1h, 2h ..) und gibt den Datensatz wieder von allen
#Daten die Binance liefert
def datagspot(df,market,timeframe,enddate):
    PATH="C:/Program Files (x86)/ChromeDriver/chromedriver.exe"
    driver = webdriver.Chrome()
    
    
    driver.get("https://data.binance.vision/?prefix=data/spot/monthly/klines/"+market+"/"+timeframe+"/")
    try:
        main=driver.find_element(By.ID, "listing")
        time.sleep(2)
    except:
        driver.quit
    first_paragraph = ""
    for i in range(len(main.text)-1, -1, -1):
        if main.text[i] == "\n":
                # Absatz gefunden
                break
        else:
            first_paragraph += main.text[i]
    
    # Umkehren der Zeichenkette
    first_paragraph = first_paragraph[::-1]
    # Speichern der ersten Spalte
    first_paragraph
    
    
    index = first_paragraph.find(timeframe+"-")
    if index != -1:
        startdate = first_paragraph[index + len(timeframe+"-"):index + len(timeframe+"-")+7]
    else:
        print("Zeichenfolge '15m-' nicht gefunden")
    
    start_date = pd.to_datetime(startdate)
    end_date = pd.to_datetime(enddate)
    dates = pd.date_range(start_date, end_date, freq="ME")

    # Alternative with list comprehension
    formatted_dates = [d.strftime("%Y-%m") for d in dates] + [enddate]
    for a in formatted_dates:
        url = 'https://data.binance.vision/data/spot/monthly/klines/'+market+'/' + timeframe +'/'+ market+'-'+timeframe+'-'+ a +'.zip'
        urllib.request.urlretrieve(url, 'Data.zip')

        zip_file = "Data.zip"

        # Zielverzeichnis zum Entpacken
        target_dir = "C:/Users/Dave/work"

        #Überprüfen ob es eine csv in dem Ordner gibt - wenn ja removen
        if (os.path.isfile(os.path.join(target_dir,"data.csv"))):
            os.remove(os.path.join(target_dir,"data.csv"))
    
        # Extrahieren der Datei
        with zipfile.ZipFile(zip_file, "r") as z:
          for filename in z.namelist():
            z.extract(filename, target_dir, f"newdata.xlsx")
        os.rename(os.path.join(target_dir, filename), os.path.join(target_dir, "data.csv"))

        dftemp=pd.read_csv("data.csv",usecols=[1,2,3,4],header=None,names=None)
       
        df= pd.concat([df, dftemp], axis=0, sort=False,ignore_index=True,join='outer')
    df=df.iloc[1:]
    df = df.iloc[:, 1:]
    df=df.reset_index(drop=True)
    df.columns=["open","high","low","close"]
    df=df.loc[df['open'] != "open"]
    df=df.reset_index(drop=True)
    return df
    